# Blog Post Summarization App

> This is a simple summarization app that scrapes the blog post from a specific URL using Beautiful Soup and then summarizes with a Hugginface pipeline 

# Dependencies

In [2]:
# installing  huggingface transformers, gradio and ipywidgets
!pip install transformers  
!pip install ipywidgets 
!pip install gradio 
!pip install beautifulsoup4

You should consider upgrading via the 'c:\users\tyeec\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\tyeec\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\tyeec\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\tyeec\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
#import gradio for UI interface
import gradio as gradio

#import the pipeline form transformers
from transformers import pipeline 

#import BeautifulSoup for webscraping
from bs4 import BeautifulSoup

#import requests to make html calls to websites
import requests

# Pipeline

In [4]:
# loading the summarization pipeline from huggingface transformers
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


# Get Post 

In [16]:
URL = "https://cryptohayes.medium.com/annihilation-4effdaf3c73f"

def get_article (url):
    #gets the inforation from the URL and stores it in the variable
    post = requests.get(url)
    
    #parses the data so we only get the html text
    parsed_data = BeautifulSoup(post.text, 'html.parser')
    
    #parses data again to retrieve only h1 and p elements
    results = parsed_data.find_all(['h1', 'p'])
    
    #gets the text from the indiviudal html elements 
    text = [result.text for result in results]  
    
    #joins them together 
    blog_post = ' '.join(text)
    
    return blog_post


In [29]:
article = get_article(URL)

# Chunking Text

In [42]:
def chunk_text (arr):
    curr = 0
    chunks = []
    
    #setting a common separator throughout text so it will be easiar to break up in sentences
    arr = arr.replace('.', '.*')
    arr = arr.replace('?', '?*')
    arr = arr.replace('!', '!*')
    sentences = arr.split('*')

    #iterates through the individual sentences
    for sentence in sentences:
        #checks if we have anything in our array 
        if (len(chunks) == 0 ):
            chunks.append(sentence.split(' ')) #appends the individual words of the sentence and not the whole thing as one
        else: 
            if (len(chunks[curr]) + len(sentence.split(' ')) <= 500):#checks if the length of the current chunk plus the new sentence will pass 500 words as the summarizer has a limit of 1024 
                chunks[curr].extend(sentence.split(' '))# adds the new sentence words individually to the previous list instead of as a entirely new list
            else: 
                curr+=1 #if the chunk will pass the limit increase the counter and move on to a new chunk
                chunks.append(sentence.split(' '))
    
    #joining back the individual words into sentences in the different chunks
    for i in range(len(chunks)):
        chunks[i] = ' '.join(chunks[i])
    
    return chunks

In [44]:
chunk_article = chunk_text (article)

8

# Summarizing

In [46]:
#summarize te individual chunks to summaries from 30 to 120 words in length
chunk_summary = summarizer(chunk_article, max_length=120, min_length=30, do_sample=False)

#joing the individdual chunks to make one final large summary
summary=' '.join([s['summary_text'] for s in chunk_summary])

summary

' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business . The broking firm of most of the retail side of this warfare ‘RobinHood’ is stealing from its poor, retail investors to give to its rich, capital backers .  Victor Niederhoffer is one of the most successful people alive today . He blew his hedge fund up in 1997 in a highly statistically improbable event, in which he sold puts that were targeted by market mechanics, rather than ‘truth’ The market is more leveraged, more volatile, more aggressive, better for types of trading and worse for investing .  Greenspans moral hazard of late 1994/1995 changed the market in a new way . Extraordinary people who were never wrong in risk-adjusted terms were carried out of them horizontally by players with leverage . Financial markets were now more poker – positioning, posi

# Interface

In [48]:
#building the function that will accept the blog post url and return the summary
def summarize_post (url):
    article = get_article(url)
    chunk_article = chunk_text(article)
    chunk_summary = summarizer(chunk_article, max_length=120, min_length=30, do_sample=False)
    summary=' '.join([s['summary_text'] for s in chunk_summary])
    return summary

In [49]:
#testing the function
summarize_post ("https://medium.com/@kellyhliu/7-steps-to-help-you-navigate-a-personal-sabbatical-18a7873b7f15")

" This is for those people who want to take time off to gain a better understanding of who they are and what they care about . This is not for those taking a break solely to relax or for professors who are on real sabbaticals — though you’re welcome to read it and see if anything applies . Make sure you have the resources to take a sabbatical . Give yourself that first month to have fun .  Hiking 300 miles of the PCT was that “something new and fun” for me . Backpacking day in and day out forced me to forget about work and slow down.  Life became simpler.  Job titles, promotions, and publications didn’t matter .  When you’re trying to figure out what you care about, you need to give yourself the space to explore . Only when you get a better idea of what gives you energy can you begin to more carefully lay down a plan and set goals . With so many paths to check out, it can feel overwhelming to keep trying new things .  What do you need to feel good in your day? Do you need people to con

In [50]:
# creating a simple gradio interafce using the tutorial posted on https://gradio.app/docs/#i_textbox
iface = gradio.Interface(fn=summarize_post, inputs=gradio.inputs.Textbox(lines=2, placeholder='Paste link to blog post you want summarized'),
                        outputs='text')

#settig share=True so that a temp public domain can be generated
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://26182.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x2105030af10>,
 'http://127.0.0.1:7860/',
 'https://26182.gradio.app')